In [3]:
import numpy as np
import os
import scipy.io
import scipy
import tensorflow as tf
from tensorflow import keras

In [4]:
DATASET_PATH = 'D:/dataset'
x_t_path  = DATASET_PATH + '/x_t'
r_t_path = DATASET_PATH + '/r_t'
r_s_path = DATASET_PATH + '/r_s'
a_authentic_path = DATASET_PATH + '/amplitudes_authentic'
doa_authentic_path = DATASET_PATH + '/doa_authentic'
doa_jamming_path = DATASET_PATH + '/doa_jamming'
doa_spoofed_path = DATASET_PATH + '/doa_spoofed'
NUM_SAMPLES = 250

In [10]:
# get the authentic, spoofed and jammed signals from r_t
def get_r_t():
    r_t = []
    for i in range(1, NUM_SAMPLES+1):
        SAMPLE_PATH = r_t_path + '/' + str(i) + '.mat'
        r_t.append(scipy.io.loadmat(SAMPLE_PATH)['r_t'])
    return r_t

# get the original signals from x_t
def get_x_t():
    x_t = []
    for i in range(1, NUM_SAMPLES+1):
        SAMPLE_PATH = x_t_path + '/' + str(i) + '.mat'
        x_t.append(scipy.io.loadmat(SAMPLE_PATH)['x_t'])
    return x_t

# get the doa_spoofed
def get_doa_spoofed():
    doa_spoofed = []
    for i in range(1, NUM_SAMPLES+1):
        SAMPLE_PATH = doa_spoofed_path + '/' + str(i) + '.mat'
        doa_spoofed.append(scipy.io.loadmat(SAMPLE_PATH)['doa_spoofed'])
    return doa_spoofed

# get the doa_jamming
def get_doa_jamming():
    doa_jamming = []
    for i in range(1, NUM_SAMPLES+1):
        SAMPLE_PATH = doa_jamming_path + '/' + str(i) + '.mat'
        doa_jamming.append(scipy.io.loadmat(SAMPLE_PATH)['doa_jamming'])
    return doa_jamming

# get the doa_authentic
def get_doa_authentic():
    doa_authentic = []
    for i in range(1, NUM_SAMPLES+1):
        SAMPLE_PATH = doa_authentic_path + '/' + str(i) + '.mat'
        doa_authentic.append(scipy.io.loadmat(SAMPLE_PATH)['doa_authentic'])
    return doa_authentic

# get the amplitudes_authentic
def get_amplitudes_authentic():
    amplitudes_authentic = []
    for i in range(1, NUM_SAMPLES+1):
        SAMPLE_PATH = a_authentic_path + '/' + str(i) + '.mat'
        amplitudes_authentic.append(scipy.io.loadmat(SAMPLE_PATH)['amplitudes_authentic'])
    return amplitudes_authentic

# get the r_s
def get_r_s():
    r_s = []
    for i in range(1, NUM_SAMPLES+1):
        SAMPLE_PATH = r_s_path + '/' + str(i) + '.mat'
        r_s.append(scipy.io.loadmat(SAMPLE_PATH)['r_s'])
    return r_s


def get_data():
    r_t = get_r_t()
    # x_t = get_x_t()
    doa_spoofed = get_doa_spoofed()
    doa_jamming = get_doa_jamming()
    doa_authentic = get_doa_authentic()
    amplitudes_authentic = get_amplitudes_authentic()
    # r_s = get_r_s()
    return r_t, doa_spoofed, doa_jamming, doa_authentic, amplitudes_authentic

# split the data into training and testing
def split_data(r_t, y):
    x_train, x_test = r_t[:200], r_t[200:]
    y_train, y_test = y[:200], y[200:]
    return x_train, x_test, y_train, y_test
    




In [6]:
r_t, doa_spoofed, doa_jamming, doa_authentic, amplitudes_authentic = get_data()
print(r_t[0].shape)
print(doa_spoofed[0].shape)
print(doa_jamming[0].shape)
print(doa_authentic[0].shape)
print(amplitudes_authentic[0].shape)

(10, 38363)
(1, 4)
(1, 1)
(1, 4)
(1, 4)


In [28]:
# combine doa_spoofed, doa_jamming, doa_authentic, amplitudes_authentic into one array
y = []
for i in range(NUM_SAMPLES):
    y.append(np.concatenate((doa_spoofed[i], doa_jamming[i], doa_authentic[i], amplitudes_authentic[i]), axis=1))
print(len(y))
print(y[0].shape)

250
(1, 13)


In [29]:
# split the data into training and testing
x_train, x_test, y_train, y_test = split_data(r_t, y)


In [67]:
# define a CNN model that takes in the r_t and y_train include dropout and batch normalization layers
# 
# each r_t is of shape (10, 38363)
# each y_train is of shape (13, 1)
def get_model():
    model = keras.Sequential()
    model.add(keras.layers.Conv1D(32, 3, activation='relu', input_shape=x_train[0].shape, data_format='channels_first'))
    model.add(keras.layers.MaxPooling1D(2))
    model.add(keras.layers.Conv1D(64, 3, activation='relu'))
    model.add(keras.layers.MaxPooling1D(2))
    model.add(keras.layers.Conv1D(128, 3, activation='relu'))
    model.add(keras.layers.MaxPooling1D(2))
    model.add(keras.layers.Conv1D(256, 3, activation='relu'))
    model.add(keras.layers.MaxPooling1D(2))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(256, activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(256, activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(y_train[0].shape[0], activation='softmax'))
    # optimiser rmsprop
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [89]:
# define a MLP model that takes in the r_t and y_train
# each r_t is of shape (10, 38363)
# each y_train is of shape (13,)
def get_mlp_model():
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(10, 38363)))
    model.add(keras.layers.Dense(512, activation='relu'))
    model.add(keras.layers.Dense(13, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [90]:
mlp_model = get_mlp_model()
mlp_model.fit(x_train, y_train, epochs=10)

ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[383630,512] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:StatelessRandomUniformV2]

In [87]:
print(x_train[0].shape)
print(y_train[0].shape)
print(len(x_train))
print(len(y_train))

# reshape x_train and y_train to have the same shape
x_train = np.reshape(x_train, (200, 383630))
y_train = np.reshape(y_train, (200, 13))


(10, 38363)
(13,)
200
200


MemoryError: Unable to allocate 1.14 GiB for an array with shape (200, 10, 38363) and data type complex128

In [88]:
model = get_model()
print(model.summary())
model.fit(np.asarray(x_train), np.asarray(y_train), epochs=10)


Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_70 (Conv1D)          (None, 32, 38361)         992       
                                                                 
 max_pooling1d_60 (MaxPoolin  (None, 16, 38361)        0         
 g1D)                                                            
                                                                 
 conv1d_71 (Conv1D)          (None, 14, 64)            7365376   
                                                                 
 max_pooling1d_61 (MaxPoolin  (None, 7, 64)            0         
 g1D)                                                            
                                                                 
 conv1d_72 (Conv1D)          (None, 5, 128)            24704     
                                                                 
 max_pooling1d_62 (MaxPoolin  (None, 2, 128)         

MemoryError: Unable to allocate 1.14 GiB for an array with shape (200, 10, 38363) and data type complex128

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

In [ ]:
model.save('model.h5')
